In [1]:
from functions import *
import pandas as pd


In [2]:
dataset = pd.read_excel('./prompt_library.xlsx',sheet_name='categorized')


In [3]:
dataset.head()
limit = 5 #no. of questions to prompt

In [4]:
# Process each row and store the results in a new DataFrame
results = []
for index, row in dataset.iterrows():
    try:
        if index >= limit:
            break
        prompt = row["Prompt"]
        answer, response_time = process_prompt(prompt)

        accuracy = evaluate_response(prompt, answer, EvaluationType.ACCURACY)
        relevance = evaluate_response(prompt, answer, EvaluationType.RELEVANCE)
        results.append((prompt,answer,response_time,accuracy,relevance))

        print(f"Question {index} processed succesfully")
    except Exception as e:
        print(f"Error Question {index} processed unsuccesfully: {e}")
        continue

Question 0 processed succesfully
Question 1 processed succesfully
Question 2 processed succesfully
Question 3 processed succesfully
Question 4 processed succesfully


In [5]:
# Create a new DataFrame from the results
output_dataset = pd.DataFrame(results, columns=['Question', 'Answer', 'Response Time (s)', 'Accuracy','Relevance'])

In [6]:
# Write the new DataFrame to a new Excel file
output_file_path = 'output_data.xlsx'
output_dataset.to_excel(output_file_path, index=False)

In [7]:
output_dataset

,Question,Answer,Response Time (s),Accuracy,Relevance
0,Can you identify the main AI and machine learn...,The documents provided highlight several AI an...,86.714674,I would rate the output a 4.\n\nThe response i...,I would rate the output a 4.\n\nThe response i...
1,How did Microsoft researchers address issues o...,Microsoft researchers addressed issues of bias...,84.063624,The response provided is accurate in summarizi...,I would rate the response a 4.\n\nThe response...
2,How did Microsoft's research at NeurIPS 2023 c...,Microsoft's research at NeurIPS 2023 contribut...,61.018703,I would rate the output a 4.\n\nThe response i...,I would rate the relevance of the output as a ...
3,How do Microsoft's papers at NeurIPS contribut...,The requested information is not available in ...,18.117342,The output is accurate. The user's question is...,1
4,How do Microsoft's NeurIPS 2023 submissions ad...,Microsoft's submissions to NeurIPS 2023 showca...,72.670112,I would rate the output a 4.\n\nThe response i...,I would rate the relevance of the output to th...


In [8]:
print(f"Processing complete. Results saved to {output_file_path}")

Processing complete. Results saved to output_data.xlsx


In [9]:
""" Compare response Evaluation """
results = []
for index, row in dataset.iterrows():
    try:
        if index >= limit:
            break
        prompt = row["Prompt"]
        answer_1, response_time_1 = process_prompt(prompt)
        prompt_paraphrased = paraphrase_question(prompt)
        answer_2, response_time_2 = process_prompt(prompt_paraphrased)
        
        evaluation = evaluate_and_compare_prompt_responses(prompt, answer_1, answer_2)
        results.append((prompt,prompt_paraphrased,answer_1,answer_2,response_time_1,response_time_2,evaluation))
        print(f"Question {index} processed succesfully")
    except Exception as e:
        print(f"Error Question {index} processed unsuccesfully: {e}")
        continue

Question 0 processed succesfully
Question 1 processed succesfully
Question 2 processed succesfully
Question 3 processed succesfully
Question 4 processed succesfully


In [10]:
comparison_output_dataset = pd.DataFrame(results, columns=['Question', 'Question Paraphrased', 'Answer 1', 'Answer 2', 'Response Time 1 (s)', 'Response Time 2 (s)', 'Evaluation',])

In [11]:
output_file_path = 'comparison_output_data.xlsx'
comparison_output_dataset.to_excel(output_file_path, index=False)

In [12]:
comparison_output_dataset

,Question,Question Paraphrased,Answer 1,Answer 2,Response Time 1 (s),Response Time 2 (s),Evaluation
0,Can you identify the main AI and machine learn...,What are the key issues in AI and machine lear...,The documents provided highlight several AI an...,"At the NeurIPS 2023 conference, Microsoft focu...",64.681616,51.131769,Response 1 is better because it provides a str...
1,How did Microsoft researchers address issues o...,In which ways did the researchers from Microso...,Microsoft researchers addressed issues of bias...,The researchers from Microsoft addressed chall...,59.620012,37.466187,Both responses provide a comprehensive overvie...
2,How did Microsoft's research at NeurIPS 2023 c...,In what ways did the research presented by Mic...,Microsoft's research contributions at NeurIPS ...,The research presented by Microsoft at the 202...,35.909462,52.501470,Response 1 is better because it directly cites...
3,How do Microsoft's papers at NeurIPS contribut...,In what ways do the contributions from Microso...,The requested information is not available in ...,The contributions from Microsoft's research pa...,15.486423,59.498719,Response 2 is better because it provides a det...
4,How do Microsoft's NeurIPS 2023 submissions ad...,What progress have Microsoft's contributions t...,Microsoft's submissions to NeurIPS 2023 showca...,Microsoft's contributions to NeurIPS 2023 have...,73.172310,73.357816,Response 1 is better because it provides a str...


In [13]:
print(f"Processing complete. Results saved to {output_file_path}")

Processing complete. Results saved to comparison_output_data.xlsx
